In [7]:
import pandas as pd
import numpy as np
import sklearn



In [28]:
data_essays = pd.read_csv('data/train_essays.csv')
data_essays_test = pd.read_csv('data/test_essays.csv')


In [26]:
data_essays

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [24]:
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [37]:
# Предполагается, что у вас есть данные, содержащие тексты и соответствующих им авторов.
# Предполагается, что вы уже предобработали тексты и собрали их в формате списка.

# Пример данных
texts = data_essays['text'].tolist()
authors = data_essays['generated'].tolist()

# # Используйте LabelEncoder для преобразования меток авторов в числа
# label_encoder = LabelEncoder()
# encoded_authors = label_encoder.fit_transform(authors)

# Обучите Word2Vec модель на ваших текстах (здесь приведен пример)
# Мы предполагаем, что тексты были предварительно обработаны (токенизированы, удалены стоп-слова и т.д.)
word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)

# Получите векторы для каждого текста, усредняя векторы его слов
def get_text_vector(text):
    vectors = [word2vec_model.wv[word] for word in text.split() if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Преобразуйте все тексты в векторы
text_vectors = [get_text_vector(text) for text in texts]

# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(text_vectors, authors, test_size=0.2, random_state=42)

# Создайте модель нейронной сети
model = Sequential()
model.add(Dense(64, input_dim=word2vec_model.vector_size, activation='relu'))
model.add(Dense(64, input_dim=word2vec_model.vector_size, activation='relu'))
model.add(Dense(len(np.unique(authors)), activation='softmax'))

# Скомпилируйте модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучите модель
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32, validation_split=0.2)

# Оцените модель
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
28/28 [==============================] - 1s 16ms/step - loss: 0.2265 - accuracy: 0.9705 - val_loss: 0.0126 - val_accuracy: 1.0000
Epoch 2/10
28/28 [==============================] - 0s 7ms/step - loss: 0.0176 - accuracy: 0.9977 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 3/10
28/28 [==============================] - 0s 8ms/step - loss: 0.0165 - accuracy: 0.9977 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 4/10
28/28 [==============================] - 0s 10ms/step - loss: 0.0163 - accuracy: 0.9977 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 5/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0160 - accuracy: 0.9977 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 6/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0158 - accuracy: 0.9977 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 7/10
28/28 [==============================] - 0s 5ms/step - loss: 0.0158 - accuracy: 0.9977 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 8/10
28/28 [

In [43]:
model.predict(np.array(X_test))

9/9 [==============================] - 0s 2ms/step


array([[9.95969296e-01, 4.03076829e-03],
       [9.96358335e-01, 3.64170084e-03],
       [9.98612106e-01, 1.38783571e-03],
       [9.99345839e-01, 6.54150266e-04],
       [9.95474994e-01, 4.52497648e-03],
       [9.97873127e-01, 2.12679105e-03],
       [9.98077035e-01, 1.92291650e-03],
       [9.97681379e-01, 2.31862254e-03],
       [9.97629046e-01, 2.37096008e-03],
       [9.99325097e-01, 6.74907002e-04],
       [9.97916222e-01, 2.08371016e-03],
       [9.98879075e-01, 1.12089608e-03],
       [9.99099851e-01, 9.00173502e-04],
       [9.99998152e-01, 1.84280498e-06],
       [9.99547660e-01, 4.52322129e-04],
       [9.95474994e-01, 4.52497881e-03],
       [9.97491360e-01, 2.50848802e-03],
       [9.95474994e-01, 4.52497648e-03],
       [9.95474994e-01, 4.52497648e-03],
       [9.97726738e-01, 2.27322918e-03],
       [9.99319971e-01, 6.80005527e-04],
       [9.99952495e-01, 4.74594526e-05],
       [9.95474994e-01, 4.52497648e-03],
       [9.99310911e-01, 6.89081382e-04],
       [9.914201

In [44]:
X_test

[array([ 0.10951783,  0.1446589 ,  0.4870757 ,  0.08363171, -0.30336443,
        -0.27820373, -0.41377753,  0.28666767,  0.36680412, -0.2523508 ,
         0.21741264, -0.37057734,  0.03371326,  0.0167233 ,  0.23147072,
        -0.1929538 ,  0.14189059,  0.2294592 , -0.30146015, -0.08808399,
         0.1087964 , -0.27341467,  0.21710613, -0.21553633, -0.12567192,
        -0.32328486,  0.16913904,  0.05884428,  0.31125444,  0.04463325,
        -0.06623495,  0.07955006, -0.08393979, -0.00845834,  0.08885956,
         0.09942271,  0.31333414, -0.11181882, -0.15100855,  0.07993621,
         0.22289358, -0.03759059, -0.10897174, -0.18249694,  0.18558316,
         0.09768014, -0.11763711, -0.13333967,  0.05013525, -0.0196759 ,
        -0.13803332, -0.4797388 ,  0.402418  , -0.01494489,  0.00292401,
        -0.03169963, -0.15608242, -0.17990302, -0.16662717,  0.05209716,
        -0.00642235,  0.04972703, -0.0888628 , -0.22959654,  0.12119637,
        -0.05331801,  0.53855044,  0.28873464, -0.1